## Notebook by John Uzoma

#### Load silver table into a dataframe

In [1]:
# Read silver tables as dataframes
textForecast = spark.read.table("lakehouse.dublintextforecast_silver")
weatherForecast = spark.read.table("lakehouse.dublinweatherforecast_silver")
weatherWarning = spark.read.table("lakehouse.dublinweatherwarning_silver")

StatementMeta(, 5653086f-7283-4c23-aad3-c591222e20c8, 3, Finished, Available, Finished)

#### Optimize delta table writes

In [2]:
# Enable V-Order
spark.conf.set("spark.sql.parquet.vorder.enabled", "true")
    
# Enable automatic Delta optimized write
spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "true")

StatementMeta(, 5653086f-7283-4c23-aad3-c591222e20c8, 4, Finished, Available, Finished)

#### Generate fact dataframe

In [3]:
# List of all columns except "Issued" from textForecast and weatherWarning
textForecast_columns_to_select = [col for col in textForecast.columns if col != "Issued"]
weatherWarning_columns_to_select = [col for col in weatherWarning.columns if col != "Issued"]

# Perform the join and select the required columns to generate fact dataframe
fact_df = weatherForecast.join(textForecast, weatherForecast["Date"] == textForecast["Issued"], "left") \
        .join(weatherWarning, weatherForecast["Date"] == weatherWarning["Issued"], "left") \
        .select(
            weatherForecast["*"], 
            *[textForecast[col] for col in textForecast_columns_to_select], 
            *[weatherWarning[col] for col in weatherWarning_columns_to_select]
        )

StatementMeta(, 5653086f-7283-4c23-aad3-c591222e20c8, 5, Finished, Available, Finished)

#### Define schema for gold fact table

In [4]:
from pyspark.sql.types import TimestampType, StringType, FloatType, DateType
from delta.tables import DeltaTable

DeltaTable.createIfNotExists(spark) \
    .tableName("lakehouse.fact_dublinweather_gold") \
    .addColumn("Date", DateType()) \
    .addColumn("Time", StringType()) \
    .addColumn("Temperature_celsius", FloatType()) \
    .addColumn("PreviousTemperature", FloatType()) \
    .addColumn("WindDirection", StringType()) \
    .addColumn("WindSpeed_mps", FloatType()) \
    .addColumn("PreviousWindSpeed", FloatType()) \
    .addColumn("WindGust_mps", FloatType()) \
    .addColumn("PreviousWindGust", FloatType()) \
    .addColumn("GlobalRadiation_wpsqm", FloatType()) \
    .addColumn("PreviousGlobalRadiation", FloatType()) \
    .addColumn("Humidity_percent", FloatType()) \
    .addColumn("PreviousHumidity", FloatType()) \
    .addColumn("Pressure_hPa", FloatType()) \
    .addColumn("PreviousPressure", FloatType()) \
    .addColumn("Cloudiness_percent", FloatType()) \
    .addColumn("PreviousCloudiness", FloatType()) \
    .addColumn("DewpointTemperature_celsius", FloatType()) \
    .addColumn("PreviousDewpointTemperature", FloatType()) \
    .addColumn("ForecastFrom", TimestampType()) \
    .addColumn("ForecastTo", TimestampType()) \
    .addColumn("Precipitation_mm", FloatType()) \
    .addColumn("PreviousPrecipitation", FloatType()) \
    .addColumn("WeatherType", StringType()) \
    .addColumn("TextForecastIssueTime", StringType()) \
    .addColumn("Today", StringType()) \
    .addColumn("Tonight", StringType()) \
    .addColumn("Tomorrow", StringType()) \
    .addColumn("Outlook", StringType()) \
    .addColumn("Level", StringType()) \
    .addColumn("Severity", StringType()) \
    .addColumn("Certainty", StringType()) \
    .addColumn("WarningIssueTime", StringType()) \
    .addColumn("Updated", TimestampType()) \
    .addColumn("Onset", TimestampType()) \
    .addColumn("Expiry", TimestampType()) \
    .addColumn("Headline", StringType()) \
    .addColumn("Description", StringType()) \
    .addColumn("Status", StringType()) \
    .execute()

StatementMeta(, 5653086f-7283-4c23-aad3-c591222e20c8, 6, Finished, Available, Finished)

#### Write fact dataframe to gold table (upsert operation)

In [5]:
# Update existing records and insert new ones based on a condition defined by the columns: Date and Time
    
deltaTable = DeltaTable.forPath(spark, 'Tables/fact_dublinweather_gold')    

dfUpdates = fact_df
    
deltaTable.alias('gold') \
  .merge(
    dfUpdates.alias('updates'),
    'gold.Date = updates.Date and gold.Time = updates.Time'
  ) \
   .whenMatchedUpdate(set =
    {
      "Temperature_celsius": "updates.Temperature_celsius",
      "PreviousTemperature": "updates.PreviousTemperature",
      "WindDirection": "updates.WindDirection",
      "WindSpeed_mps": "updates.WindSpeed_mps",
      "PreviousWindSpeed": "updates.PreviousWindSpeed",
      "WindGust_mps": "updates.WindGust_mps",
      "PreviousWindGust": "updates.PreviousWindGust",
      "GlobalRadiation_wpsqm": "updates.GlobalRadiation_wpsqm",
      "PreviousGlobalRadiation": "updates.PreviousGlobalRadiation",
      "Humidity_percent": "updates.Humidity_percent",
      "PreviousHumidity": "updates.PreviousHumidity",
      "Pressure_hPa": "updates.Pressure_hPa",
      "PreviousPressure": "updates.PreviousPressure",
      "Cloudiness_percent": "updates.Cloudiness_percent",
      "PreviousCloudiness": "updates.PreviousCloudiness",
      "DewpointTemperature_celsius": "updates.DewpointTemperature_celsius",
      "PreviousDewpointTemperature": "updates.PreviousDewpointTemperature",
      "ForecastFrom": "updates.ForecastFrom",
      "ForecastTo": "updates.ForecastTo",
      "Precipitation_mm": "updates.Precipitation_mm",
      "PreviousPrecipitation": "updates.PreviousPrecipitation",
      "WeatherType": "updates.WeatherType",
      "TextForecastIssueTime": "updates.TextForecastIssueTime",
      "Today": "updates.Today",
      "Tonight": "updates.Tonight",
      "Tomorrow": "updates.Tomorrow",
      "Outlook": "updates.Outlook",
      "Level": "updates.Level",
      "Severity": "updates.Severity",
      "Certainty": "updates.Certainty",
      "WarningIssueTime": "updates.WarningIssueTime",
      "Updated": "updates.Updated",
      "Onset": "updates.Onset",
      "Expiry": "updates.Expiry",
      "Headline": "updates.Headline",
      "Description": "updates.Description",
      "Status": "updates.Status"
    }
  ) \
 .whenNotMatchedInsert(values =
    {
      "Date": "updates.Date",
      "Time": "updates.Time",
      "Temperature_celsius": "updates.Temperature_celsius",
      "PreviousTemperature": "updates.PreviousTemperature",
      "WindDirection": "updates.WindDirection",
      "WindSpeed_mps": "updates.WindSpeed_mps",
      "PreviousWindSpeed": "updates.PreviousWindSpeed",
      "WindGust_mps": "updates.WindGust_mps",
      "PreviousWindGust": "updates.PreviousWindGust",
      "GlobalRadiation_wpsqm": "updates.GlobalRadiation_wpsqm",
      "PreviousGlobalRadiation": "updates.PreviousGlobalRadiation",
      "Humidity_percent": "updates.Humidity_percent",
      "PreviousHumidity": "updates.PreviousHumidity",
      "Pressure_hPa": "updates.Pressure_hPa",
      "PreviousPressure": "updates.PreviousPressure",
      "Cloudiness_percent": "updates.Cloudiness_percent",
      "PreviousCloudiness": "updates.PreviousCloudiness",
      "DewpointTemperature_celsius": "updates.DewpointTemperature_celsius",
      "PreviousDewpointTemperature": "updates.PreviousDewpointTemperature",
      "ForecastFrom": "updates.ForecastFrom",
      "ForecastTo": "updates.ForecastTo",
      "Precipitation_mm": "updates.Precipitation_mm",
      "PreviousPrecipitation": "updates.PreviousPrecipitation",
      "WeatherType": "updates.WeatherType",
      "TextForecastIssueTime": "updates.TextForecastIssueTime",
      "Today": "updates.Today",
      "Tonight": "updates.Tonight",
      "Tomorrow": "updates.Tomorrow",
      "Outlook": "updates.Outlook",
      "Level": "updates.Level",
      "Severity": "updates.Severity",
      "Certainty": "updates.Certainty",
      "WarningIssueTime": "updates.WarningIssueTime",
      "Updated": "updates.Updated",
      "Onset": "updates.Onset",
      "Expiry": "updates.Expiry",
      "Headline": "updates.Headline",
      "Description": "updates.Description",
      "Status": "updates.Status"
    }
  ) \
  .execute()

StatementMeta(, 5653086f-7283-4c23-aad3-c591222e20c8, 7, Finished, Available, Finished)

#### Generate date dimension dataframe

In [6]:
from pyspark.sql.functions import dayofmonth, year, date_format

# Extract date, day, month and year values from fact_df
date_dim_df = fact_df.select(
        ("Date"), \
        dayofmonth("Date").alias("Day"), \
        date_format("Date", "MMM").substr(1, 3).alias("Month"), \
        year("Date").alias("Year")
    ).orderBy("Date")

StatementMeta(, 5653086f-7283-4c23-aad3-c591222e20c8, 8, Finished, Available, Finished)

#### Drop duplicates in Date column

In [7]:
date_dim_df = date_dim_df.dropDuplicates(["Date"])

StatementMeta(, 5653086f-7283-4c23-aad3-c591222e20c8, 9, Finished, Available, Finished)

#### Define schema for date dimension table

In [8]:
from pyspark.sql.types import IntegerType
from delta.tables import DeltaTable

# Define the schema for the dim_date_gold table
DeltaTable.createIfNotExists(spark) \
    .tableName("lakehouse.dim_date_gold") \
    .addColumn("Date", DateType()) \
    .addColumn("Day", IntegerType()) \
    .addColumn("Month", StringType()) \
    .addColumn("Year", IntegerType()) \
    .execute()

StatementMeta(, 5653086f-7283-4c23-aad3-c591222e20c8, 10, Finished, Available, Finished)

#### Write date dimension dataframe to gold table (upsert operation)

In [9]:
# Update existing records and insert new ones based on a condition defined by the column: Date
    
deltaTable = DeltaTable.forPath(spark, 'Tables/dim_date_gold')    

dfUpdates = date_dim_df
    
deltaTable.alias('gold') \
  .merge(
    dfUpdates.alias('updates'),
    'gold.Date = updates.Date'
  ) \
   .whenMatchedUpdate(set =
    {
      "Day": "updates.Day",
      "Month": "updates.Month",
      "Year": "updates.Year"
    }
  ) \
 .whenNotMatchedInsert(values =
    {
      "Date": "updates.Date",
      "Day": "updates.Day",
      "Month": "updates.Month",
      "Year": "updates.Year"
    }
  ) \
  .execute()

StatementMeta(, 5653086f-7283-4c23-aad3-c591222e20c8, 11, Finished, Available, Finished)

#### Generate weather type dimension dataframe

In [11]:
from pyspark.sql.functions import dayofmonth, year, date_format, when, lower

# Extract WeatherType from fact dataframe and drop duplicates
weather_type_dim_df = fact_df.select(("WeatherType")).dropDuplicates(["WeatherType"])

# Create column Flag to have value 1 when WeatherType contains rain or drizzle (case3 insensitive), otherwise 0
weather_type_dim_df = weather_type_dim_df.withColumn(
    "Flag", 
    when(
        lower(weather_type_dim_df.WeatherType).contains("rain") | lower(weather_type_dim_df.WeatherType).contains("drizzle"),
        1
    ).otherwise(0)
)

StatementMeta(, 5653086f-7283-4c23-aad3-c591222e20c8, 13, Finished, Available, Finished)

#### Define schema for weather type dimension table

In [12]:
from pyspark.sql.types import ByteType

DeltaTable.createIfNotExists(spark) \
    .tableName("lakehouse.dim_weathertype_gold") \
    .addColumn("WeatherType", StringType()) \
    .addColumn("Flag", ByteType()) \
    .execute()

StatementMeta(, 5653086f-7283-4c23-aad3-c591222e20c8, 14, Finished, Available, Finished)

#### Write weather type dimension dataframe to gold table (overwrite operation)

In [16]:
weather_type_dim_df.write.format("delta").mode("overwrite").option("mergeSchema", "true").save("Tables/dim_weathertype_gold")

StatementMeta(, 5653086f-7283-4c23-aad3-c591222e20c8, 18, Finished, Available, Finished)